## Start date : July 10th, 2023 (Data Validation of 2022 Business Data)

Author: Kenthia

- Goals: 

1. Use Dask to go through the 2022 original unfiltered dataset to see what stores are there and what's missing.
2. Figure out if there are other variables we could use from the larger dataset to apply to our filtered dataset to answer questions like : Was Costco filtered correctly ? 

Extra: Rework the string matching 


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import dask.dataframe as dd
import glob
import os

In [2]:
#reading in data in pandas 
data = pd.read_csv("/srv/data/my_shared_data_folder/rafi_2022/2022_business00.txt",sep=",", nrows=100000)
data

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MIRAGE DRAPERIES,4731 E SUNNY DUNES RD,PALM SPRINGS,CA,92264,1536.0,65.0,760,2,A,A,225903,CURTAINS-MANUFACTURERS,31324003.0,KNIT FABRIC MILLS,571407.0,DRAPERIES & CURTAINS-RETAIL/CUSTOM MADE,259103.0,WINDOW SHADES-MANUFACTURERS,259198.0,DRAPERY HDWR-WINDOW BLINDS/SHADES (MFRS),502328.0,WINDOW SHADES-WHOLESALE,519953.0,WINDOW SHADES EQUIP & SUPLS-WHOLESALE,2022,NaN,1.0,70.0,9,NaN,NaN,NaN,NaN,491039277,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,44804.0,1.0,33.812119,-116.494500,0,40140.0,2.0,348.0,6065.0
1,NaN,11700 OLIO RD,FISHERS,IN,46037,7618.0,57.0,317,1,A,A,804907,NURSES-PRACTITIONERS,62139923.0,OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,1.0,60.0,9,NaN,NaN,NaN,NaN,747806590,NaN,NaN,NaN,NaN,NaN,NaN,258751049.0,NaN,7,110808.0,1.0,39.959999,-85.921268,P,26900.0,2.0,294.0,18057.0
2,STONE RESIN SURFACING LLC,1 MAYFLOWER PL,MILFORD,CT,6460,4520.0,9.0,203,2,D,F,282105,RESINS-MANUFACTURERS,32521108.0,PLASTICS MATERIAL & RESIN MANUFACTURING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,25.0,10452.0,9,NaN,2018.0,NaN,NaN,738467458,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,150200.0,2.0,41.207739,-73.070136,P,35300.0,2.0,408.0,9009.0
3,NaN,2121 7TH ST,PARKERSBURG,WV,26101,3803.0,107.0,740,1,NaN,NaN,832221,SOCIAL WORKERS,62133003.0,OFFICES-MENTAL HEALTH PRACTITIONERS (EXC PHYS...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,762964650,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,702.0,1.0,39.269405,-81.520599,P,37620.0,2.0,425.0,54107.0
4,IRONWOOD STATE PRISON,19005 WILEYS WELL RD,BLYTHE,CA,92225,2287.0,65.0,760,2,A,NaN,912102,GOVERNMENT OFFICES-STATE,92112008.0,LEGISLATIVE BODIES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,922302.0,STATE GOVT-CORRECTIONAL INSTITUTIONS,NaN,NaN,NaN,NaN,NaN,NaN,2022,38406.0,1.0,0.0,2,NaN,NaN,NaN,NaN,493833396,NaN,475961991.0,2000.0,0.0,I,NaN,493833396.0,P,7,46900.0,1.0,33.570065,-114.898031,0,40140.0,2.0,348.0,6065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,U-HAUL NEIGHBORHOOD DEALER,46020 MANEKIN PLZ # 135,STERLING,VA,20166,6639.0,107.0,703,2,A,C,751303,TRUCK RENTING & LEASING,53212016.0,"TRUCK, UTILITY TRAILER & RV RENTAL & LEASING",751903.0,TRAILER RENTING & LEASING,751304.0,MOVING-SELF-SERVICE,751910.0,TRAILERS-TRUCK-RENTING & LEASING,NaN,NaN,NaN,NaN,2022,NaN,2.0,1039.0,2,6,NaN,NaN,NaN,758338005,200014587.0,409125614.0,1500.0,5739747.0,I,K,NaN,NaN,5,611501.0,2.0,39.023985,-77.406929,4,47900.0,2.0,548.0,51107.0
99996,ROOM TO SPARE LLC,425 SETTLERS TRACE BLVD,LAFAYETTE,LA,70508,6048.0,55.0,337,2,NaN,NaN,999977,NaN,99999004.0,UNCLASSIFIED ESTABLISHMENTS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,NaN,NaN,0.0,9,NaN,NaN,NaN,NaN,415158882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,1407.0,2.0,30.164224,-92.045185,P,29180.0,2.0,318.0,22055.0
99997,LOUISIANA ENERGY & POWER AUTH,201 3RD ST,RAYNE,LA,70578,5207.0,1.0,337,2,A,A,171131,ENERGY MANAGEMENT SYSTEMS & PRODUCTS,23822013.0,PLUMBING HTG & AIR-CONDITIONING CONTRACTORS,874212.0,ENE

In [8]:
GROCERY_NAICS= "44511"

In [9]:
df = dd.read_csv("/srv/data/my_shared_data_folder/rafi_2022/2022_business00.txt",sep=",",assume_missing=True)
df.head()

,COMPANY,ADDRESS LINE 1,CITY,STATE,ZIPCODE,ZIP4,COUNTY CODE,AREA CODE,IDCODE,LOCATION EMPLOYEE SIZE CODE,LOCATION SALES VOLUME CODE,PRIMARY SIC CODE,SIC6_DESCRIPTIONS,PRIMARY NAICS CODE,NAICS8 DESCRIPTIONS,SIC CODE,SIC6_DESCRIPTIONS (SIC),SIC CODE 1,SIC6_DESCRIPTIONS (SIC1),SIC CODE 2,SIC6_DESCRIPTIONS(SIC2),SIC CODE 3,SIC6_DESCRIPTIONS(SIC3),SIC CODE 4,SIC6_DESCRIPTIONS(SIC4),ARCHIVE VERSION YEAR,YELLOW PAGE CODE,EMPLOYEE SIZE (5) - LOCATION,SALES VOLUME (9) - LOCATION,BUSINESS STATUS CODE,INDUSTRY SPECIFIC FIRST BYTE,YEAR ESTABLISHED,OFFICE SIZE CODE,COMPANY HOLDING STATUS,ABI,SUBSIDIARY NUMBER,PARENT NUMBER,PARENT ACTUAL EMPLOYEE SIZE,PARENT ACTUAL SALES VOLUME,PARENT EMPLOYEE SIZE CODE,PARENT SALES VOLUME CODE,SITE NUMBER,ADDRESS TYPE INDICATOR,POPULATION CODE,CENSUS TRACT,CENSUS BLOCK,LATITUDE,LONGITUDE,MATCH CODE,CBSA CODE,CBSA LEVEL,CSA CODE,FIPS CODE
0,MIRAGE DRAPERIES,4731 E SUNNY DUNES RD,PALM SPRINGS,CA,92264.0,1536.0,65.0,760.0,2.0,A,A,225903.0,CURTAINS-MANUFACTURERS,31324003.0,KNIT FABRIC MILLS,571407.0,DRAPERIES & CURTAINS-RETAIL/CUSTOM MADE,259103.0,WINDOW SHADES-MANUFACTURERS,259198.0,DRAPERY HDWR-WINDOW BLINDS/SHADES (MFRS),502328.0,WINDOW SHADES-WHOLESALE,519953.0,WINDOW SHADES EQUIP & SUPLS-WHOLESALE,2022.0,NaN,1.0,70.0,9.0,NaN,NaN,NaN,NaN,491039277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,44804.0,1.0,33.812119,-116.494500,0,40140.0,2.0,348.0,6065.0
1,NaN,11700 OLIO RD,FISHERS,IN,46037.0,7618.0,57.0,317.0,1.0,A,A,804907.0,NURSES-PRACTITIONERS,62139923.0,OFFICES OF ALL OTHER MISC HEALTH PRACTITIONERS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,NaN,1.0,60.0,9.0,NaN,NaN,NaN,NaN,747806590.0,NaN,NaN,NaN,NaN,NaN,NaN,258751049.0,NaN,7.0,110808.0,1.0,39.959999,-85.921268,P,26900.0,2.0,294.0,18057.0
2,STONE RESIN SURFACING LLC,1 MAYFLOWER PL,MILFORD,CT,6460.0,4520.0,9.0,203.0,2.0,D,F,282105.0,RESINS-MANUFACTURERS,32521108.0,PLASTICS MATERIAL & RESIN MANUFACTURING,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,NaN,25.0,10452.0,9.0,NaN,2018.0,NaN,NaN,738467458.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,150200.0,2.0,41.207739,-73.070136,P,35300.0,2.0,408.0,9009.0
3,NaN,2121 7TH ST,PARKERSBURG,WV,26101.0,3803.0,107.0,740.0,1.0,NaN,NaN,832221.0,SOCIAL WORKERS,62133003.0,OFFICES-MENTAL HEALTH PRACTITIONERS (EXC PHYS...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,762964650.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,702.0,1.0,39.269405,-81.520599,P,37620.0,2.0,425.0,54107.0
4,IRONWOOD STATE PRISON,19005 WILEYS WELL RD,BLYTHE,CA,92225.0,2287.0,65.0,760.0,2.0,A,NaN,912102.0,GOVERNMENT OFFICES-STATE,92112008.0,LEGISLATIVE BODIES,999966.0,FEDERAL GOVERNMENT CONTRACTORS,922302.0,STATE GOVT-CORRECTIONAL INSTITUTIONS,NaN,NaN,NaN,NaN,NaN,NaN,2022.0,38406.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,493833396.0,NaN,475961991.0,2000.0,0.0,I,NaN,493833396.0,P,7.0,46900.0,1.0,33.570065,-114.898031,0,40140.0,2.0,348.0,6065.0


In [10]:
filtered_df = df.loc[["PRIMARY NAICS CODE"] == "44511"]
result = filtered_df.compute() # dask requires you to run a "compute" step to actually complete the filtering operation


ParserError: Error tokenizing data. C error: EOF inside string starting at row 170998